In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("/content/drive/MyDrive/players_data_full_realistic.csv")

target_player_id = df.iloc[59]["Player_ID"]
target_player = df[df["Player_ID"] == target_player_id].iloc[0]

filtered_df = df[df["Sport"] == target_player["Sport"]].copy()
filtered_df = filtered_df[filtered_df["Player_ID"] != target_player_id]

numeric_features = ["Age", "Hours_Available"]
categorical_features = [
    "City", "Neighborhood", "Skill_Level", "Competitive_Level",
    "Available_Days", "Available_Times"
]

scaler = MinMaxScaler()
scaled_numeric = scaler.fit_transform(filtered_df[numeric_features])

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded_cats = encoder.fit_transform(filtered_df[categorical_features])

full_data = np.hstack([scaled_numeric, encoded_cats])

target_numeric = scaler.transform([target_player[numeric_features]])
target_cats = encoder.transform([target_player[categorical_features]])
target_full = np.hstack([target_numeric, target_cats])

similarities = cosine_similarity(target_full.reshape(1, -1), full_data)[0]
filtered_df["Similarity"] = similarities

top_matches = filtered_df.sort_values(by="Similarity", ascending=False).head(5)

print("أفضل المنافسين للاعب:", target_player["Player_Name"])
print(top_matches[[
    "Player_ID", "Player_Name", "Sport", "Skill_Level", "Neighborhood",
    "Available_Days", "Available_Times", "Similarity"
]])


أفضل المنافسين للاعب: أحمد الهاجري
     Player_ID    Player_Name Sport Skill_Level Neighborhood  \
8569  ID_18569   تركي الخثعمي  بادل       مبتدئ   بطحاء قريش   
4895  ID_14895     ريم الحربي  بادل       مبتدئ      الشوقية   
1276  ID_11276   ماجد الخثعمي  بادل       مبتدئ     العقربية   
8620  ID_18620  جواهر العسيري  بادل       محترف       الزاهر   
5921  ID_15921    ناصر العنزي  بادل       مبتدئ    الجامعيين   

                Available_Days Available_Times  Similarity  
8569                  الأربعاء          صباحًا    0.706306  
4895   الخميس, السبت, الثلاثاء   صباحًا, مساءً    0.702520  
1276           الأحد, الأربعاء          صباحًا    0.575058  
8620  الأحد, الاثنين, الثلاثاء          صباحًا    0.575058  
5921   الأربعاء, الأحد, الخميس          صباحًا    0.572986  


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/MyDrive/players_data_full_realistic.csv")

target_player_id = df.iloc[59]["Player_ID"]
target_player = df[df["Player_ID"] == target_player_id].iloc[0]

filtered_df = df[(df["Sport"] == target_player["Sport"]) &
                 (df["Gender"] == target_player["Gender"]) &
                 (df["City"] == target_player["City"])].copy()
filtered_df = filtered_df[filtered_df["Player_ID"] != target_player_id]

numeric_features = ["Age", "Hours_Available"]
categorical_features = [
    "City", "Neighborhood", "Skill_Level", "Competitive_Level",
    "Available_Days", "Available_Times", "Gender"
]

scaler = MinMaxScaler()
scaled_numeric = scaler.fit_transform(filtered_df[numeric_features])

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded_cats = encoder.fit_transform(filtered_df[categorical_features])

full_data = np.hstack([scaled_numeric, encoded_cats])

target_numeric = scaler.transform([target_player[numeric_features]])
target_cats = encoder.transform([target_player[categorical_features]])
target_full = np.hstack([target_numeric, target_cats])

X = full_data
y = np.ones(full_data.shape[0])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

predictions = model.predict(X)

filtered_df["Predicted_Similarity"] = predictions

top_matches = filtered_df.sort_values(by="Predicted_Similarity", ascending=False).head(5)

print("أفضل المنافسين للاعب:", target_player["Player_Name"])
print(top_matches[[
    "Player_ID", "Player_Name", "Sport", "Skill_Level", "City", "Neighborhood",
    "Available_Days", "Available_Times", "Gender", "Predicted_Similarity"
]])


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 502ms/step - accuracy: 0.5021 - loss: 0.6928 - val_accuracy: 1.0000 - val_loss: 0.6454
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step - accuracy: 0.9354 - loss: 0.6403 - val_accuracy: 1.0000 - val_loss: 0.5907
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 1.0000 - loss: 0.5828 - val_accuracy: 1.0000 - val_loss: 0.5305
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 1.0000 - loss: 0.5334 - val_accuracy: 1.0000 - val_loss: 0.4680
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 1.0000 - loss: 0.4591 - val_accuracy: 1.0000 - val_loss: 0.4028
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 1.0000 - loss: 0.3934 - val_accuracy: 1.0000 - val_loss: 0.3365
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 0.3252 - val_accuracy: 1.0000 - val_loss: 0.2721
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 0.2699 - val_accuracy: 1.0000 - val_loss: 0.2111
Epoch